### Importando bibliotecas

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

import pandas as pd
from io import StringIO

### URL de alguns indices da B3

In [2]:
# Ibovespa
url_ibov = 'https://sistemaswebb3-listados.b3.com.br/indexPage/day/IBOV?language=pt-br'
# Ibovespa + BDR
url_ibovplus = 'https://sistemaswebb3-listados.b3.com.br/indexPage/day/IBBR?language=pt-br'
# Ibovespa Amplo
url_ibramplo = 'https://sistemaswebb3-listados.b3.com.br/indexPage/day/IBRA?language=pt-br'
# IBRX 100
url_ibrx100 = 'https://sistemaswebb3-listados.b3.com.br/indexPage/day/IBXX?language=pt-br'
# IBRX 50
url_ibrx50 = 'https://sistemaswebb3-listados.b3.com.br/indexPage/day/IBXL?language=pt-br'
# fundos imobiliários
url_ifix = 'https://sistemaswebb3-listados.b3.com.br/indexPage/day/IFIX?language=pt-br'

### Captura de dados e tratamento do DataFrame

In [7]:
driver = webdriver.Chrome()

# acessando url
driver.get(url_ibov)

# determinando implicitly wait de 10 segundos
driver.implicitly_wait(time_to_wait= 10)

# mudando página de 20 para 120 
select_page = driver.find_element(By.ID, 'selectPage')
option_120 = driver.find_element(By.XPATH, '//*[@id="selectPage"]/option[contains(text(), "120")]')
ActionChains(driver).move_to_element(select_page).pause(1).click(select_page).pause(1).perform()
option_120.click()
ActionChains(driver).pause(2).perform()

dados_coletados = []

while True:
    try:

        # capturando dados da tabela e transformando em DataFrame
        df = pd.read_html(io= StringIO(driver.page_source))
        df = df[0]

        # tratando dados do DataFrame
        # Retira as últimas duas linhas
        df = df[:-2]
        # Transformando os valores percentuais
        df['Part. (%)'] = df['Part. (%)'].apply(lambda x: float(str(x).replace(',', '.'))) / 100000

        # adicionando DataFrame a lista vazia
        dados_coletados.append(df)

        # próxima página
        proxima_pagina = driver.find_element(By.CSS_SELECTOR, 'li > a[aria-label=" page"]')
        proxima_pagina.click()

    except:
        break

# Concatenando DataFrames
df_final = pd.concat(dados_coletados)

# fechando o driver
driver.close()

In [10]:
df.head()

,Código,Ação,Tipo,Qtde. Teórica,Part. (%)
0,ALOS3,ALLOS,ON NM,502.481.592,0.00520
1,ALPA4,ALPARGATAS,PN N1,166.460.180,0.00054
2,ABEV3,AMBEV S/A,ON,4.394.835.131,0.02624
3,ASAI3,ASSAI,ON NM,1.349.687.675,0.00542
4,AURE3,AUREN,ON NM,291.727.616,0.00147


In [8]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Código         86 non-null     object 
 1   Ação           86 non-null     object 
 2   Tipo           86 non-null     object 
 3   Qtde. Teórica  86 non-null     object 
 4   Part. (%)      86 non-null     float64
dtypes: float64(1), object(4)
memory usage: 3.5+ KB


In [9]:
df_final['Part. (%)'].sum()

0.9999999999999999

### Salvando DataFrame

In [ ]:
# # salvando em csv
# df.to_csv('df.csv', index= False)